In [2]:
import os
import pandas as pd
import string
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
data=pd.read_csv(r"D:\Data Science\Datasets\twcs_sample.csv")

In [4]:
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [5]:
data.tail()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0
92,119335,Tesco,False,Wed Oct 11 15:38:07 +0000 2017,@105861 If that doesn't help please DM your fu...,NaN,119333.0


# Text Processing

## Remove HTML tags 

In [6]:
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text

In [7]:
data['text']=data['text'].apply(remove_html_tags)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


## Remove URLs

In [8]:
def remove_urls(text):
    url_pattern = r'https?://\S+|www\.\S+'
    clean_text = re.sub(url_pattern, '', text)
    return clean_text

In [9]:
data['text']=data['text'].apply(remove_urls)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - or...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


## Convert to lowercase

In [10]:
data['text']=data['text'].str.lower()
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@applesupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 your business means a lot to us. pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 i really hope you all change but i'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 livechat is online at the moment - or...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@virgintrains see attached error message. i've...,119243,119240.0


## Remove emojis

In [11]:
def remove_emoji(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    clean_text = re.sub(emoji_pattern, '', text)
    return clean_text

In [12]:
data['text']=data['text'].apply(remove_emoji)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@applesupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 your business means a lot to us. pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 i really hope you all change but i'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 livechat is online at the moment - or...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@virgintrains see attached error message. i've...,119243,119240.0


## Remove punctuation 

In [13]:
def remove_punc(text):
    exclude = set(string.punctuation)
    for char in exclude:
        text = text.replace(char, "")
    return text

In [14]:
data['text']=data['text'].apply(remove_punc)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport causing the reply to be disregard...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 your business means a lot to us please ...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 i really hope you all change but im sure...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat is online at the moment or c...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrains see attached error message ive tr...,119243,119240.0


## Remove Stopwords

In [15]:
def remove_stopwords(text, language='english'):
    stop_words = set(stopwords.words(language))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_text)
    return filtered_text

In [16]:
data['text']=data['text'].apply(remove_stopwords)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport causing reply disregarded tapped ...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 business means lot us please dm name zi...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 really hope change im sure wont dont,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat online moment contact 03331 03...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrains see attached error message ive tr...,119243,119240.0


## Handle abbreviations 

In [17]:
abbr_dict={
  "AFAIK": "As Far As I Know",
  "AFK": "Away From Keyboard",
  "ASAP": "As Soon As Possible",
  "ATK": "At The Keyboard",
  "ATM": "At The Moment",
  "A3": "Anytime, Anywhere, Anyplace",
  "BAK": "Back At Keyboard",
  "BBL": "Be Back Later",
  "BBS": "Be Back Soon",
  "BFN": "Bye For Now",
  "B4N": "Bye For Now",
  "BRB": "Be Right Back",
  "BRT": "Be Right There",
  "BTW": "By The Way",
  "B4": "Before",
  "CU": "See You",
  "CUL8R": "See You Later",
  "CYA": "See You",
  "FAQ": "Frequently Asked Questions",
  "FC": "Fingers Crossed",
  "FWIW": "For What It's Worth",
  "FYI": "For Your Information",
  "GAL": "Get A Life",
  "GG": "Good Game",
  "GN": "Good Night",
  "GMTA": "Great Minds Think Alike",
  "GR8": "Great!",
  "G9": "Genius",
  "IC": "I See",
  "ICQ": "I Seek you (also a chat program)",
  "ILU": "I Love You",
  "IMHO": "In My Honest/Humble Opinion",
  "IMO": "In My Opinion",
  "IOW": "In Other Words",
  "IRL": "In Real Life",
  "KISS": "Keep It Simple, Stupid",
  "LDR": "Long Distance Relationship",
  "LMAO": "Laugh My A.. Off",
  "LOL": "Laughing Out Loud",
  "LTNS": "Long Time No See",
  "L8R": "Later",
  "MTE": "My Thoughts Exactly",
  "M8": "Mate",
  "NRN": "No Reply Necessary",
  "OIC": "Oh I See",
  "PITA": "Pain In The A..",
  "PRT": "Party",
  "PRW": "Parents Are Watching",
  "QPSA?": "Que Pasa?",
  "ROFL": "Rolling On The Floor Laughing",
  "ROFLOL": "Rolling On The Floor Laughing Out Loud",
  "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
  "SK8": "Skate",
  "STATS": "Your sex and age",
  "ASL": "Age, Sex, Location",
  "THX": "Thank You",
  "TTFN": "Ta-Ta For Now!",
  "TTYL": "Talk To You Later",
  "U": "You",
  "U2": "You Too",
  "U4E": "Yours For Ever",
  "WB": "Welcome Back",
  "WTF": "What The F...",
  "WTG": "Way To Go!",
  "WUF": "Where Are You From?",
  "W8": "Wait...",
  "7K": "Sick:-D Laugher",
  "TFW": "That feeling when. TFW internet slang often goes in a caption to an image.",
  "MFW": "My face when",
  "MRW": "My reaction when",
  "IFYP": "I feel your pain",
  "TNTL": "Trying not to laugh",
  "JK": "Just kidding",
  "IDC": "I don’t care",
  "ILY": "I love you",
  "IMU": "I miss you",
  "ADIH": "Another day in hell",
  "ZZZ": "Sleeping, bored, tired",
  "WYWH": "Wish you were here",
  "TIME": "Tears in my eyes",
  "BAE": "Before anyone else",
  "FIMH": "Forever in my heart",
  "BSAAW": "Big smile and a wink",
  "BWL": "Bursting with laughter",
  "BFF": "Best friends forever",
  "CSL": "Can’t stop laughing"
}

In [18]:
def abbr_text(text):
    words = text.split()
    expanded_words = [abbr_dict.get(word.lower(), word) for word in words]
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [19]:
data['text']=data['text'].apply(abbr_text)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport causing reply disregarded tapped ...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 business means lot us please dm name zi...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 really hope change im sure wont dont,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat online moment contact 03331 03...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrains see attached error message ive tr...,119243,119240.0


## Stemming 

In [20]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def stem_text(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in tokens]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [21]:
data['text']=data['text'].apply(stem_text)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport caus repli disregard tap notif ke...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 busi mean lot us pleas dm name zip code...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 realli hope chang im sure wont dont,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat onlin moment contact 03331 031...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrain see attach error messag ive tri le...,119243,119240.0


## Spelling correction 

In [22]:
from textblob import TextBlob
def spell_check_text(text):
    blob = TextBlob(text)
    corrected_text = str(blob.correct())
    return corrected_text

In [23]:
data['text']=data['text'].apply(spell_check_text)
data

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport cause reply disregard tap notify ...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 busy mean lot us pleas do name zip code...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 really hope change in sure wont dont,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat online moment contact 03331 03...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrain see attach error message give try ...,119243,119240.0
...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,105860 wish amazon option get ship up store av...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,reschedul shit tomorrow,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,105861 hey sera sorry hear issue ask lay speed...,119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,test bit find layout cumbersom remove item hav...,"119335,119336",119332.0


## Remove Whitespace

In [24]:
def whitespace_clean(text):
    clean_text = text.strip()
    return clean_text


In [25]:
data['text']=data['text'].apply(whitespace_clean)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,applesupport cause reply disregard tap notify ...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,105835 busy mean lot us pleas do name zip code...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,76328 really hope change in sure wont dont,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,105836 livechat online moment contact 03331 03...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,virgintrain see attach error message give try ...,119243,119240.0


## Tokenize 

In [26]:
from nltk.tokenize import word_tokenize, sent_tokenize
def tokenize_texts(input_series):
    tokenized_data = input_series.apply(lambda text: (sent_tokenize(text), word_tokenize(text)))
    return tokenized_data

In [27]:
data['text']=tokenize_texts(data['text'])
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,([applesupport cause reply disregard tap notif...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,([105835 busy mean lot us pleas do name zip co...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,"([76328 really hope change in sure wont dont],...",119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,([105836 livechat online moment contact 03331 ...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,([virgintrain see attach error message give tr...,119243,119240.0


## One-Hot Encoding (OHE)

In [34]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Extracting text from the tuples
texts = [item[0] for item in data['text']]

# Initialize the OneHotEncoder
encoder = OneHotEncoder()
ohe_matrix = encoder.fit_transform(np.array(texts).reshape(-1, 1))

In [48]:
encoder.categories_

[array(['105676 understand frustrate situate pleas follow us send do assist spring care ac',
        '105835 busy mean lot us pleas do name zip code admit detail concern or',
        '105836 livechat online moment contact 03331 031 031 option 1 4 3 leave message request call back',
        '105836 that miriam team send email shortly he',
        '105836 try not devil miriam mm',
        '105836 work ok miriam link help mm',
        '105837 help version to find set it genet it reply do',
        '105838 applesupport suffer hope find slut',
        '105839 thank reach us away happy help send us do look together',
        '105840 hi devil happen could also let us know andros spotifi version your use that great ay',
        '105840 thank distance could possible affect playback log it start devil it log back make differ ay',
        '105840 that great hear any come let us know well carry help ay',
        '105840 your welcome there any else help give us shout up',
        '105841 wed love h

## Bag of Words (BOW) 

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

# Custom preprocessor function to lowercase the text
def custom_preprocessor(text):
    return ' '.join([doc.lower() for doc in text])

# Extracting text from the tuples
texts = [item[0] for item in data['text']]

# Initialize CountVectorizer with custom preprocessor
vectorizer = CountVectorizer(preprocessor=custom_preprocessor)

# Fit and transform the text data
bow_matrix = vectorizer.fit_transform(texts)


In [45]:
vectorizer.vocabulary_

{'applesupport': 82,
 'cause': 119,
 'reply': 380,
 'disregard': 165,
 'tap': 446,
 'notify': 318,
 'keyboard': 258,
 'open': 329,
 '105835': 4,
 'busy': 112,
 'mean': 292,
 'lot': 284,
 'us': 480,
 'pleas': 349,
 'do': 167,
 'name': 310,
 'zip': 507,
 'code': 130,
 'admit': 63,
 'detail': 157,
 'concern': 135,
 'or': 331,
 '76328': 48,
 'really': 371,
 'hope': 232,
 'change': 121,
 'in': 240,
 'sure': 440,
 'wont': 501,
 'dont': 170,
 '105836': 5,
 'livechat': 277,
 'online': 327,
 'moment': 304,
 'contact': 137,
 '03331': 1,
 '031': 0,
 'option': 330,
 'leave': 268,
 'message': 294,
 'request': 381,
 'call': 116,
 'back': 97,
 'virgintrain': 484,
 'see': 390,
 'attach': 92,
 'error': 181,
 'give': 208,
 'try': 469,
 'voicemail': 486,
 'never': 313,
 'time': 462,
 'past': 342,
 'week': 490,
 'not': 316,
 'devil': 158,
 'miriam': 300,
 'mm': 301,
 'yep': 504,
 'lawton': 265,
 'today': 464,
 'differ': 160,
 'brother': 110,
 'work': 502,
 'ok': 323,
 'link': 274,
 'help': 226,
 'still': 

## N-gram Vectorization (Line 2,3,4)

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
def custom_preprocessor(text):
    return ' '.join([doc.lower() for doc in text])
# Extracting text from the tuples
texts = [item[0] for item in data['text']]

# Initialize CountVectorizer with ngram_range
ngram_vectorizer = CountVectorizer(preprocessor=custom_preprocessor,ngram_range=(2,2))

# Fit and transform the text data
ngram_matrix = ngram_vectorizer.fit_transform(texts)


In [56]:
ngram_vectorizer.vocabulary_

{'applesupport cause': 111,
 'cause reply': 172,
 'reply disregard': 601,
 'disregard tap': 232,
 'tap notify': 710,
 'notify keyboard': 509,
 'keyboard open': 415,
 '105835 busy': 5,
 'busy mean': 158,
 'mean lot': 475,
 'lot us': 464,
 'us pleas': 798,
 'pleas do': 554,
 'do name': 239,
 'name zip': 500,
 'zip code': 865,
 'code admit': 181,
 'admit detail': 83,
 'detail concern': 215,
 'concern or': 191,
 '76328 really': 68,
 'really hope': 591,
 'hope change': 373,
 'change in': 174,
 'in sure': 388,
 'sure wont': 702,
 'wont dont': 851,
 '105836 livechat': 6,
 'livechat online': 447,
 'online moment': 523,
 'moment contact': 490,
 'contact 03331': 193,
 '03331 031': 2,
 '031 031': 0,
 '031 option': 1,
 'option leave': 527,
 'leave message': 432,
 'message request': 478,
 'request call': 603,
 'call back': 163,
 'virgintrain see': 823,
 'see attach': 615,
 'attach error': 131,
 'error message': 262,
 'message give': 477,
 'give try': 319,
 'try leave': 766,
 'leave voicemail': 433,

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
def custom_preprocessor(text):
    return ' '.join([doc.lower() for doc in text])
# Extracting text from the tuples
texts = [item[0] for item in data['text']]

# Initialize CountVectorizer with ngram_range
ngram_vectorizer = CountVectorizer(preprocessor=custom_preprocessor,ngram_range=(3,3))

# Fit and transform the text data
ngram_matrix = ngram_vectorizer.fit_transform(texts)
ngram_vectorizer.vocabulary_

{'applesupport cause reply': 107,
 'cause reply disregard': 164,
 'reply disregard tap': 573,
 'disregard tap notify': 221,
 'tap notify keyboard': 672,
 'notify keyboard open': 483,
 '105835 busy mean': 5,
 'busy mean lot': 153,
 'mean lot us': 450,
 'lot us pleas': 440,
 'us pleas do': 762,
 'pleas do name': 527,
 'do name zip': 229,
 'name zip code': 474,
 'zip code admit': 825,
 'code admit detail': 173,
 'admit detail concern': 81,
 'detail concern or': 207,
 '76328 really hope': 67,
 'really hope change': 564,
 'hope change in': 357,
 'change in sure': 166,
 'in sure wont': 371,
 'sure wont dont': 664,
 '105836 livechat online': 6,
 'livechat online moment': 426,
 'online moment contact': 497,
 'moment contact 03331': 464,
 'contact 03331 031': 183,
 '03331 031 031': 2,
 '031 031 option': 0,
 '031 option leave': 1,
 'option leave message': 500,
 'leave message request': 410,
 'message request call': 452,
 'request call back': 574,
 'virgintrain see attach': 785,
 'see attach erro

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
def custom_preprocessor(text):
    return ' '.join([doc.lower() for doc in text])
# Extracting text from the tuples
texts = [item[0] for item in data['text']]

# Initialize CountVectorizer with ngram_range
ngram_vectorizer = CountVectorizer(preprocessor=custom_preprocessor,ngram_range=(4,4))

# Fit and transform the text data
ngram_matrix = ngram_vectorizer.fit_transform(texts)
ngram_vectorizer.vocabulary_

{'applesupport cause reply disregard': 107,
 'cause reply disregard tap': 159,
 'reply disregard tap notify': 532,
 'disregard tap notify keyboard': 211,
 'tap notify keyboard open': 617,
 '105835 busy mean lot': 5,
 'busy mean lot us': 151,
 'mean lot us pleas': 415,
 'lot us pleas do': 407,
 'us pleas do name': 692,
 'pleas do name zip': 490,
 'do name zip code': 217,
 'name zip code admit': 439,
 'zip code admit detail': 751,
 'code admit detail concern': 167,
 'admit detail concern or': 81,
 '76328 really hope change': 67,
 'really hope change in': 524,
 'hope change in sure': 330,
 'change in sure wont': 161,
 'in sure wont dont': 343,
 '105836 livechat online moment': 6,
 'livechat online moment contact': 395,
 'online moment contact 03331': 460,
 'moment contact 03331 031': 429,
 'contact 03331 031 031': 177,
 '03331 031 031 option': 2,
 '031 031 option leave': 0,
 '031 option leave message': 1,
 'option leave message request': 463,
 'leave message request call': 376,
 'message 

## TF-IDF Transformation:

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

def custom_preprocessor(text):
    return ' '.join([doc.lower() for doc in text])

texts = [item[0] for item in data['text']]


tfidf_vectorizer = TfidfVectorizer(preprocessor=custom_preprocessor)
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
tfidf_vectorizer.vocabulary_

{'applesupport': 82,
 'cause': 119,
 'reply': 380,
 'disregard': 165,
 'tap': 446,
 'notify': 318,
 'keyboard': 258,
 'open': 329,
 '105835': 4,
 'busy': 112,
 'mean': 292,
 'lot': 284,
 'us': 480,
 'pleas': 349,
 'do': 167,
 'name': 310,
 'zip': 507,
 'code': 130,
 'admit': 63,
 'detail': 157,
 'concern': 135,
 'or': 331,
 '76328': 48,
 'really': 371,
 'hope': 232,
 'change': 121,
 'in': 240,
 'sure': 440,
 'wont': 501,
 'dont': 170,
 '105836': 5,
 'livechat': 277,
 'online': 327,
 'moment': 304,
 'contact': 137,
 '03331': 1,
 '031': 0,
 'option': 330,
 'leave': 268,
 'message': 294,
 'request': 381,
 'call': 116,
 'back': 97,
 'virgintrain': 484,
 'see': 390,
 'attach': 92,
 'error': 181,
 'give': 208,
 'try': 469,
 'voicemail': 486,
 'never': 313,
 'time': 462,
 'past': 342,
 'week': 490,
 'not': 316,
 'devil': 158,
 'miriam': 300,
 'mm': 301,
 'yep': 504,
 'lawton': 265,
 'today': 464,
 'differ': 160,
 'brother': 110,
 'work': 502,
 'ok': 323,
 'link': 274,
 'help': 226,
 'still': 

# Advantage and Disadvantages

| Technique            | Advantages                                                                                              | Disadvantages                                                                                                      |
|----------------------|---------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------|
| One-Hot Encoding (OHE) | - Simple and intuitive representation.   Preserves individual word information.  Can handle categorical features easily.  Suitable for small categorical feature spaces.  Works well with various machine learning algorithms. | - High dimensionality, especially for large vocabularies.   Sparse representation leading to memory inefficiency.   Doesn't capture semantic relationships between words.   Unable to handle out-of-vocabulary words.   Doesn't account for word frequency. |
| Bag of Words (BOW)     | - Simple and straightforward to implement.  Captures word frequencies in a document.  Can be effective for text classification tasks.  Works well with linear models.  Easy to interpret and analyze. | - Doesn't consider word order or context.   Treats all words equally, ignoring their importance.   Vocabulary size can be large, leading to a high-dimensional space.   Not suitable for capturing semantic meaning.   Unable to handle out-of-vocabulary words. |
| N-gram Vectorization   | - Captures some contextual information through word sequences.  Can provide better performance than BOW for some tasks.  Preserves some local word order information.  More flexible in capturing phrases and idioms.  Can enhance the performance of models in some cases. | - Increased dimensionality with larger n-grams.   Requires more memory and computational resources.   Limited ability to capture long-range dependencies.   Vulnerable to data sparsity issues with rare n-grams.   Still unable to capture global semantic meaning. |
| TF-IDF Transformation | - Considers both term frequency and inverse document frequency.  Reduces the impact of common words while emphasizing rare ones.  Provides better representation for documents than simple word counts.  Helps to identify important words in a document collection.  Can improve the performance of text classification and clustering tasks. | - Loss of word order and context information.   Doesn't capture relationships between words within a document.   Limited in handling short documents effectively.   Requires careful tuning of parameters for optimal performance.   Not suitable for capturing syntactic or semantic meaning. |
